In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from domain_model import Scenario
from simulation import ACC, acc_parameters, hdm_parameters, SimulationLeadBraking, HDM
from stats import KDE, kde_from_file
from databaseemulator import DataBaseEmulator
%matplotlib inline

# Test driver model, ACC, and CACC

In [ ]:
s = SimulationLeadBraking(follower=HDM(), follower_parameters=hdm_parameters)

In [ ]:
np.random.seed(0)
s.simulation(dict(v0=20, amean=1.72155613, dv=18.159153), plot=True, seed=0)

In [ ]:
s2 = SimulationLeadBraking(follower=ACC(), follower_parameters=acc_parameters)
np.random.seed(0)
s2.simulation(dict(v0=20, amean=2, dv=15), plot=True)

In [ ]:
kde = kde_from_file(os.path.join("data", "6_kde", "lead_braking.p"))
kde.set_bandwidth(kde.bandwidth)
kde2d = kde_from_file(os.path.join("data", "6_kde", "lead_braking_2d.p"))
kde2d.set_bandwidth(kde2d.bandwidth)
np.random.seed(0)
x = kde.conditional_sample(i_given=0, values=20, n_samples=1000)
k = KDE(data=x, scaling=True)
k.set_bandwidth(kde2d.bandwidth)

In [ ]:
INIT_SPEED = 20
def plot_2d_kde(k, amean_lim=(0., 3.), dv_lim=(0., 20.)):
    scaling_kde = (k.cdf(np.array([[400., dv_lim[1]]]))[0]-
                   k.cdf(np.array([[400., dv_lim[0]]]))[0]-
                   k.cdf(np.array([[amean_lim[0], dv_lim[1]]]))[0]+
                   k.cdf(np.array([[amean_lim[0], dv_lim[0]]]))[0])
    AMEANS, DVS = np.meshgrid(np.linspace(amean_lim[0], amean_lim[1], 100),
                              np.linspace(dv_lim[0], dv_lim[1], 50))
    pdf = k.score_samples(np.concatenate((AMEANS[:, :, np.newaxis],
                                          DVS[:, :, np.newaxis]), axis=2))
    FIG, AXES = plt.subplots(1, 1)
    CONTOUR = AXES.contourf(AMEANS, DVS, pdf/scaling_kde)
    FIG.colorbar(CONTOUR)
    AXES.set_xlabel("Mean deceleration [m/s$^2$]")
    AXES.set_ylabel("Speed difference [m/s]")
    AXES.set_title("Probability density")
    return pdf/scaling_kde
pdf = plot_2d_kde(k)

# Monte Carlo

In [ ]:
np.random.seed(0)
n = 1000
x_mc = np.zeros((n, 2))
r_mc = np.zeros(n)
for i in tqdm(range(n)):
    while True:
        sample = kde2d.sample()
        amean, vdiff = sample[0][0], sample[0][1]
        if amean > 0 and 0 < vdiff <= 20:
            break
    x_mc[i] = [amean, vdiff]
    r_mc[i] = s.simulation(dict(v0=20, amean=amean, dv=vdiff))
    i += 1

In [ ]:
prob = np.mean(r_mc < 0)
sigma = np.sqrt(np.mean(((r_mc < 0) - prob)**2) / n)
print("Probability of collision: {:.2f} %".format(prob*100), end="")
print(" +/- {:.2f} %".format(sigma*100))

In [ ]:
plt.scatter(x_mc[:, 0], x_mc[:, 1], c=(r_mc<0))

# Importance sampling with MCMC

In [ ]:
n_is = 2000
np.random.seed(0)
x_is, r_is = np.zeros((n_is, 2)), np.zeros(n_is)
istart = -1
amean, vdiff = 2, 20
result = s.simulation(dict(v0=20, amean=amean, dv=vdiff))
std_amean = 0.3
std_vdiff = 2.0
current_prob = kde2d.score_samples(np.array([[amean, vdiff]]))

np.random.seed(0)
for i in range(istart, n_is):
    amean_candidate = amean + np.random.randn()*std_amean
    vdiff_candidate = vdiff + np.random.randn()*std_vdiff
    if vdiff_candidate > 20 or vdiff_candidate <= 0 or amean_candidate <= 0:
        candidate_prob = 0
    else:
        candidate_prob = kde2d.score_samples(np.array([[amean_candidate, vdiff_candidate]]))
    prob_mcmc = np.random.rand()
    if prob_mcmc < candidate_prob/current_prob:
        # In this case, if there is a collision, this is a good candidate.
        result_candidate = s.simulation(dict(v0=20, amean=amean_candidate, dv=vdiff_candidate))
        if result_candidate < 0:        
            amean = amean_candidate
            vdiff = vdiff_candidate
            current_prob = candidate_prob
            result = result_candidate

    if i >= 0:
        x_is[i] = [amean, vdiff]
        r_is[i] = result

In [ ]:
np.unique(r_is).shape

In [ ]:
ktmp = KDE(x_is, scaling=True)
ktmp.clustering()
ktmp.compute_bandwidth()
ktmp.bandwidth

In [ ]:
pdf = plot_2d_kde(ktmp, amean_lim=(1.0, 3.7), dv_lim=(15.5, 21.0))
plt.scatter(ktmp.data[:, 0] * ktmp.data_helpers.std[0], 
            ktmp.data[:, 1] * ktmp.data_helpers.std[1],
            c=ktmp.data_helpers.weights)

In [ ]:
def get_scaling(k):
    scaling_kde = (k.cdf(np.array([[400., 20.0]]))[0]-
                   k.cdf(np.array([[400., 0.0]]))[0]-
                   k.cdf(np.array([[0.0, 20.0]]))[0]+
                   k.cdf(np.array([[0.0, 0.0]]))[0])
    return scaling_kde

In [ ]:
importance_density = ktmp.score_samples(x_is) / get_scaling(ktmp)
orig_density = kde2d.score_samples(x_is) / get_scaling(kde2d)

In [ ]:
prob = np.mean(orig_density / importance_density)
sigma = np.sqrt(np.mean((orig_density / importance_density - prob)**2) / n_is)
print("Probability of collision: {:.2f} %".format(prob*100), end="")
print(" +/- {:.2f} %".format(sigma*100))

# Other stuff

In [ ]:
x = pd.read_csv(os.path.join("data", "7_simulation_results", "lead_braking_is_2d.csv"))

In [ ]:
np.unique(x[["amean", "dv"]].values, axis=0, return_counts=True)

In [ ]:
x.loc[50:]